<a href="https://colab.research.google.com/github/hamidahoderinwale/model_metadata_analyses/blob/main/scraping_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Script 3
!pip install huggingface_hub
!pip install adapters
import requests
import pandas as pd
import datetime
import json
import csv
from huggingface_hub import HfApi
from bs4 import BeautifulSoup
from adapters import list_adapters
from huggingface_hub import hf_hub_download
from adapters import AutoAdapterModel
import re



# Initialize API
api = HfApi()

# Function to validate Hugging Face model URL
def validate_hf_model_url(url):
    pattern = r"^https://huggingface.co/([\w\-]+)/([\w\-]+)$"
    match = re.match(pattern, url)
    return match.groups() if match else None

# page with model finetunes
def get_finetuned_models_page(model_org, model_name):
    all_model_links = []  # Store all links across pages
    page_num = 0
    while True:
        search_url = f"https://huggingface.co/models?other=base_model:finetune:{model_org}/{model_name}&p={page_num}"
        response = requests.get(search_url)
        if response.status_code != 200:
            break  # Exit if page not found

        soup = BeautifulSoup(response.text, "html.parser")
        model_divs = soup.find_all("div", class_="w-full truncate")
        if not model_divs:
            break  # Exit if no more models on the page

        for div in model_divs:
            header = div.find("header")
            if header:
                model_link = header.get("title")
                if model_link:
                    all_model_links.append(f"https://huggingface.co/{model_link}")

        page_num += 1  # Move to the next page

    return all_model_links

# model card data
def get_model_card(model_id):
    try:
        # Try to download model card if available
        readme_path = hf_hub_download(repo_id=model_id, filename='README.md')
        with open(readme_path, 'r', encoding='utf-8') as f:
            card_content = f.read()
        return card_content
    except Exception as e:
        print(f"  Could not download model card: {str(e)[:100]}...")
        return ""


# Function to get parent model
def get_parent_model(model_url):
    return model_url.split(f"/{model_name}/")[0]

# Truncate metadata
def filter_metadata(json_metadata):
            keys_to_keep = ["modelId", "sha", "tags", "downloads", "pipeline_tag"]
            return {k: json_metadata.get(k) for k in keys_to_keep if k in json_metadata}
            filtered_metadata = filter_metadata(api.model_info(model_id).__dict__)

# Get adapter models
def get_adapter_models_page(model_org, model_name):
    all_adapter_links = []  # Store all adapter links across pages
    page_num = 0
    while True:
        search_url = f"https://huggingface.co/models?other=base_model:adapter:{model_org}/{model_name}&p={page_num}"
        response = requests.get(search_url)
        if response.status_code != 200:
            break  # Exit if page not found

        soup = BeautifulSoup(response.text, "html.parser")
        model_divs = soup.find_all("div", class_="w-full truncate")
        if not model_divs:
            break  # Exit if no more models on the page

        for div in model_divs:
            header = div.find("header")
            if header:
                model_link = header.get("title")
                if model_link:
                    all_adapter_links.append(f"https://huggingface.co/{model_link}")

        page_num += 1  # Move to the next page

    return all_adapter_links

# Recursive DFS (depth-first search) for finding fine-tunes
def dfs_finetunes(model_url, visited, depth=0, results=None):
       if results is None:
           results = []

       if model_url in visited:
           return results
       visited.add(model_url)

       validated = validate_hf_model_url(model_url)
       if not validated:
           print(f"Invalid URL skipped: {model_url}")
           model_url = "N/A"
           return results

       model_org, model_name = validated
       model_id = f"{model_org}/{model_name}"


       print(f"\n{'  ' * depth}Fetching metadata for: {model_id}")
       try:
           model_metadata = api.model_info(model_id).__dict__
           json_metadata = json.dumps(model_metadata, default=str)
           model_card = get_model_card(model_id)

       except Exception as e:
           print(f"Error fetching metadata: {e}")
           return results

       finetune_links = get_finetuned_models_page(model_org, model_name)
       # Removing Duplicate Children
       finetune_links = list(set(finetune_links))
       print(f"{'  ' * depth}Found {len(finetune_links)} fine-tunes at depth {depth}.")
       adapter_links = get_adapter_models_page(model_org, model_name)
       print(f"{'  ' * depth}Found {len(adapter_links)} adapter models for {model_id}.")

       results.append({
           "model_id": model_id,
           "card": model_card,
           "metadata": json_metadata,
           "depth": depth,
           "children": finetune_links,
           "children_count": len(finetune_links),
           "adapters": adapter_links,
           "adapters_count": len(adapter_links),
           "parent_model": get_parent_model(model_url)
       })

       for link in finetune_links:
             dfs_finetunes(link, visited, depth + 1, results)
       return results

# Timestamp for the run
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Function to save results as JSON
def save_json(results, model_name):
    filename = f"{model_name}_finetunes_{timestamp}.json"
    data = {
        "models": results
    }
    with open(filename, "w") as f:
        json.dump(data, f, indent=4, default=str)
    print(f"Results saved to {filename}")

# Function to save results as CSV
''' def save_csv(results, model_name):
    filename = f"{model_name}_{timestamp}_finetunes.csv"
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["model_id", "depth", "children_count", "children", "metadata"])
        writer.writeheader()
        for entry in results:
            # Ensure metadata is a JSON string
            if isinstance(entry["metadata"], dict):
                entry["metadata"] = json.dumps(entry["metadata"], indent=2, default=str)
            # Join children list as a string
            entry["children"] = ", ".join(entry["children"])
            writer.writerow(entry)
    print(f"Results saved to {filename}") '''

# Function to save results as CSV (pandas)
def save_csv(results, model_name):
    filename = f"{model_name}_finetunes_{timestamp}.csv"
    df = pd.DataFrame(results)
    df.to_csv(filename, index=True)
    print(f"Results saved to {filename}")

# Main execution
if __name__ == "__main__":
    model_url = input("Enter the Hugging Face model URL: ").strip()
    visited = set()
    results = dfs_finetunes(model_url, visited)

    if results:
        model_name = results[0]["model_id"].split("/")[-1]  # Extract model name for file naming
        save_json(results, model_name)
        save_csv(results, model_name)
    else:
        print("No fine-tuned models found.")

'''Links for testing: https://huggingface.co/NousResearch/DeepHermes-3-Mistral-24B-Preview (3 fine-tunes at depth 0, 1 fine-tune at depth 1 for 'AlSamCur123/DeepHermes-3-Mistral-24BContinuedFine')
https://huggingface.co/perplexity-ai/r1-1776 (11 fine-tunes at depth 0)'''

Enter the Hugging Face model URL: https://huggingface.co/deepseek-ai/DeepSeek-R1

Fetching metadata for: deepseek-ai/DeepSeek-R1
Found 297 fine-tunes at depth 0.
Found 121 adapter models for deepseek-ai/DeepSeek-R1.


NameError: name 'model_name' is not defined

In [ ]:
# Script 3
!pip install huggingface_hub
!pip install adapters
import requests
import pandas as pd
import datetime
import json
import csv
from huggingface_hub import HfApi
from bs4 import BeautifulSoup
from adapters import list_adapters
from huggingface_hub import hf_hub_download
from adapters import AutoAdapterModel
import re



# Initialize API
api = HfApi()

# Function to validate Hugging Face model URL
def validate_hf_model_url(url):
    pattern = r"^https://huggingface.co/([\w\-]+)/([\w\-]+)$"
    match = re.match(pattern, url)
    return match.groups() if match else None

# page with model finetunes
def get_finetuned_models_page(model_org, model_name):
    all_model_links = []  # Store all links across pages
    page_num = 0
    while True:
        search_url = f"https://huggingface.co/models?other=base_model:finetune:{model_org}/{model_name}&p={page_num}"
        response = requests.get(search_url)
        if response.status_code != 200:
            break  # Exit if page not found

        soup = BeautifulSoup(response.text, "html.parser")
        model_divs = soup.find_all("div", class_="w-full truncate")
        if not model_divs:
            break  # Exit if no more models on the page

        for div in model_divs:
            header = div.find("header")
            if header:
                model_link = header.get("title")
                if model_link:
                    all_model_links.append(f"https://huggingface.co/{model_link}")

        page_num += 1  # Move to the next page

    return all_model_links

# model card data
def get_model_card(model_id):
    try:
        # Try to download model card if available
        readme_path = hf_hub_download(repo_id=model_id, filename='README.md')
        with open(readme_path, 'r', encoding='utf-8') as f:
            card_content = f.read()
        return card_content
    except Exception as e:
        print(f"  Could not download model card: {str(e)[:100]}...")
        return ""


# Function to get parent model
def get_parent_model(model_url):
    return model_url.split(f"/{model_name}/")[0]

# Truncate metadata
def filter_metadata(json_metadata):
            keys_to_keep = ["modelId", "sha", "tags", "downloads", "pipeline_tag"]
            return {k: json_metadata.get(k) for k in keys_to_keep if k in json_metadata}
            filtered_metadata = filter_metadata(api.model_info(model_id).__dict__)

# Get adapter models
def get_adapter_models_page(model_org, model_name):
    all_adapter_links = []  # Store all adapter links across pages
    page_num = 0
    while True:
        search_url = f"https://huggingface.co/models?other=base_model:adapter:{model_org}/{model_name}&p={page_num}"
        response = requests.get(search_url)
        if response.status_code != 200:
            break  # Exit if page not found

        soup = BeautifulSoup(response.text, "html.parser")
        model_divs = soup.find_all("div", class_="w-full truncate")
        if not model_divs:
            break  # Exit if no more models on the page

        for div in model_divs:
            header = div.find("header")
            if header:
                model_link = header.get("title")
                if model_link:
                    all_adapter_links.append(f"https://huggingface.co/{model_link}")

        page_num += 1  # Move to the next page

    return all_adapter_links

# Recursive DFS (depth-first search) for finding fine-tunes
def dfs_finetunes(model_url, visited, depth=0, results=None):
       if results is None:
           results = []

       if model_url in visited:
           return results
       visited.add(model_url)

       validated = validate_hf_model_url(model_url)
       if not validated:
           print(f"Invalid URL skipped: {model_url}")
           model_url = "N/A"
           return results

       model_org, model_name = validated
       model_id = f"{model_org}/{model_name}"


       print(f"\n{'  ' * depth}Fetching metadata for: {model_id}")
       try:
           model_metadata = api.model_info(model_id).__dict__
           json_metadata = json.dumps(model_metadata, default=str)
           model_card = get_model_card(model_id)

       except Exception as e:
           print(f"Error fetching metadata: {e}")
           return results

       finetune_links = get_finetuned_models_page(model_org, model_name)
       # Removing Duplicate Children
       finetune_links = list(set(finetune_links))
       print(f"{'  ' * depth}Found {len(finetune_links)} fine-tunes at depth {depth}.")
       adapter_links = get_adapter_models_page(model_org, model_name)
       print(f"{'  ' * depth}Found {len(adapter_links)} adapter models for {model_id}.")

       results.append({
           "model_id": model_id,
           "card": model_card,
           "metadata": json_metadata,
           "depth": depth,
           "children": finetune_links,
           "children_count": len(finetune_links),
           "adapters": adapter_links,
           "adapters_count": len(adapter_links),
           "parent_model": get_parent_model(model_url)
       })

       for link in finetune_links:
             dfs_finetunes(link, visited, depth + 1, results)
       return results

# Timestamp for the run
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Function to save results as JSON
def save_json(results, model_name):
    filename = f"{model_name}_finetunes_{timestamp}.json"
    data = {
        "models": results
    }
    with open(filename, "w") as f:
        json.dump(data, f, indent=4, default=str)
    print(f"Results saved to {filename}")

# Function to save results as CSV
''' def save_csv(results, model_name):
    filename = f"{model_name}_{timestamp}_finetunes.csv"
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["model_id", "depth", "children_count", "children", "metadata"])
        writer.writeheader()
        for entry in results:
            # Ensure metadata is a JSON string
            if isinstance(entry["metadata"], dict):
                entry["metadata"] = json.dumps(entry["metadata"], indent=2, default=str)
            # Join children list as a string
            entry["children"] = ", ".join(entry["children"])
            writer.writerow(entry)
    print(f"Results saved to {filename}") '''

# Function to save results as CSV (pandas)
def save_csv(results, model_name):
    filename = f"{model_name}_finetunes_{timestamp}.csv"
    df = pd.DataFrame(results)
    df.to_csv(filename, index=True)
    print(f"Results saved to {filename}")

# Main execution
if __name__ == "__main__":
    model_url = input("Enter the Hugging Face model URL: ").strip()
    visited = set()
    results = dfs_finetunes(model_url, visited)

    if results:
        model_name = results[0]["model_id"].split("/")[-1]  # Extract model name for file naming
        save_json(results, model_name)
        save_csv(results, model_name)
    else:
        print("No fine-tuned models found.")

'''Links for testing: https://huggingface.co/NousResearch/DeepHermes-3-Mistral-24B-Preview (3 fine-tunes at depth 0, 1 fine-tune at depth 1 for 'AlSamCur123/DeepHermes-3-Mistral-24BContinuedFine')
https://huggingface.co/perplexity-ai/r1-1776 (11 fine-tunes at depth 0)'''

Enter the Hugging Face model URL: https://huggingface.co/deepseek-ai/DeepSeek-R1

Fetching metadata for: deepseek-ai/DeepSeek-R1
Found 297 fine-tunes at depth 0.
Found 121 adapter models for deepseek-ai/DeepSeek-R1.


NameError: name 'model_name' is not defined

In [ ]:
# Script 3
!pip install huggingface_hub
!pip install adapters
import requests
import pandas as pd
import datetime
import json
import csv
from huggingface_hub import HfApi
from bs4 import BeautifulSoup
from adapters import list_adapters
from huggingface_hub import hf_hub_download
from adapters import AutoAdapterModel
import re



# Initialize API
api = HfApi()

# Function to validate Hugging Face model URL
def validate_hf_model_url(url):
    pattern = r"^https://huggingface.co/([\w\-]+)/([\w\-]+)$"
    match = re.match(pattern, url)
    return match.groups() if match else None

# page with model finetunes
def get_finetuned_models_page(model_org, model_name):
    all_model_links = []  # Store all links across pages
    page_num = 0
    while True:
        search_url = f"https://huggingface.co/models?other=base_model:finetune:{model_org}/{model_name}&p={page_num}"
        response = requests.get(search_url)
        if response.status_code != 200:
            break  # Exit if page not found

        soup = BeautifulSoup(response.text, "html.parser")
        model_divs = soup.find_all("div", class_="w-full truncate")
        if not model_divs:
            break  # Exit if no more models on the page

        for div in model_divs:
            header = div.find("header")
            if header:
                model_link = header.get("title")
                if model_link:
                    all_model_links.append(f"https://huggingface.co/{model_link}")

        page_num += 1  # Move to the next page

    return all_model_links

# model card data
def get_model_card(model_id):
    try:
        # Try to download model card if available
        readme_path = hf_hub_download(repo_id=model_id, filename='README.md')
        with open(readme_path, 'r', encoding='utf-8') as f:
            card_content = f.read()
        return card_content
    except Exception as e:
        print(f"  Could not download model card: {str(e)[:100]}...")
        return ""


# Function to get parent model
def get_parent_model(model_url):
    return model_url.split(f"/{model_name}/")[0]

# Truncate metadata
def filter_metadata(json_metadata):
            keys_to_keep = ["modelId", "sha", "tags", "downloads", "pipeline_tag"]
            return {k: json_metadata.get(k) for k in keys_to_keep if k in json_metadata}
            filtered_metadata = filter_metadata(api.model_info(model_id).__dict__)

# Get adapter models
def get_adapter_models_page(model_org, model_name):
    all_adapter_links = []  # Store all adapter links across pages
    page_num = 0
    while True:
        search_url = f"https://huggingface.co/models?other=base_model:adapter:{model_org}/{model_name}&p={page_num}"
        response = requests.get(search_url)
        if response.status_code != 200:
            break  # Exit if page not found

        soup = BeautifulSoup(response.text, "html.parser")
        model_divs = soup.find_all("div", class_="w-full truncate")
        if not model_divs:
            break  # Exit if no more models on the page

        for div in model_divs:
            header = div.find("header")
            if header:
                model_link = header.get("title")
                if model_link:
                    all_adapter_links.append(f"https://huggingface.co/{model_link}")

        page_num += 1  # Move to the next page

    return all_adapter_links

# Recursive DFS (depth-first search) for finding fine-tunes
def dfs_finetunes(model_url, visited, depth=0, results=None):
       if results is None:
           results = []

       if model_url in visited:
           return results
       visited.add(model_url)

       validated = validate_hf_model_url(model_url)
       if not validated:
           print(f"Invalid URL skipped: {model_url}")
           model_url = "N/A"
           return results

       model_org, model_name = validated
       model_id = f"{model_org}/{model_name}"


       print(f"\n{'  ' * depth}Fetching metadata for: {model_id}")
       try:
           model_metadata = api.model_info(model_id).__dict__
           json_metadata = json.dumps(model_metadata, default=str)
           model_card = get_model_card(model_id)

       except Exception as e:
           print(f"Error fetching metadata: {e}")
           return results

       finetune_links = get_finetuned_models_page(model_org, model_name)
       # Removing Duplicate Children
       finetune_links = list(set(finetune_links))
       print(f"{'  ' * depth}Found {len(finetune_links)} fine-tunes at depth {depth}.")
       adapter_links = get_adapter_models_page(model_org, model_name)
       print(f"{'  ' * depth}Found {len(adapter_links)} adapter models for {model_id}.")

       results.append({
           "model_id": model_id,
           "card": model_card,
           "metadata": json_metadata,
           "depth": depth,
           "children": finetune_links,
           "children_count": len(finetune_links),
           "adapters": adapter_links,
           "adapters_count": len(adapter_links),
           "parent_model": get_parent_model(model_url)
       })

       for link in finetune_links:
             dfs_finetunes(link, visited, depth + 1, results)
       return results

# Timestamp for the run
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Function to save results as JSON
def save_json(results, model_name):
    filename = f"{model_name}_finetunes_{timestamp}.json"
    data = {
        "models": results
    }
    with open(filename, "w") as f:
        json.dump(data, f, indent=4, default=str)
    print(f"Results saved to {filename}")

# Function to save results as CSV
''' def save_csv(results, model_name):
    filename = f"{model_name}_{timestamp}_finetunes.csv"
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["model_id", "depth", "children_count", "children", "metadata"])
        writer.writeheader()
        for entry in results:
            # Ensure metadata is a JSON string
            if isinstance(entry["metadata"], dict):
                entry["metadata"] = json.dumps(entry["metadata"], indent=2, default=str)
            # Join children list as a string
            entry["children"] = ", ".join(entry["children"])
            writer.writerow(entry)
    print(f"Results saved to {filename}") '''

# Function to save results as CSV (pandas)
def save_csv(results, model_name):
    filename = f"{model_name}_finetunes_{timestamp}.csv"
    df = pd.DataFrame(results)
    df.to_csv(filename, index=True)
    print(f"Results saved to {filename}")

# Main execution
if __name__ == "__main__":
    model_url = input("Enter the Hugging Face model URL: ").strip()
    visited = set()
    results = dfs_finetunes(model_url, visited)

    if results:
        model_name = results[0]["model_id"].split("/")[-1]  # Extract model name for file naming
        save_json(results, model_name)
        save_csv(results, model_name)
    else:
        print("No fine-tuned models found.")

'''Links for testing: https://huggingface.co/NousResearch/DeepHermes-3-Mistral-24B-Preview (3 fine-tunes at depth 0, 1 fine-tune at depth 1 for 'AlSamCur123/DeepHermes-3-Mistral-24BContinuedFine')
https://huggingface.co/perplexity-ai/r1-1776 (11 fine-tunes at depth 0)'''

Enter the Hugging Face model URL: https://huggingface.co/deepseek-ai/DeepSeek-R1

Fetching metadata for: deepseek-ai/DeepSeek-R1
Found 297 fine-tunes at depth 0.
Found 121 adapter models for deepseek-ai/DeepSeek-R1.


NameError: name 'model_name' is not defined

# Script 1
Takes in a (validated) model url and output its metadata

In [37]:
# Script 1: takes input model url, validates url, and gives model metadata
!pip install validators
from huggingface_hub import HfApi
import huggingface_hub as hf
import validators
import json
import csv

hf_api = HfApi()

def input_url():
    while True:
        input_model_url = input("Enter model URL: ")
        print(f"You entered: {input_model_url}")

        if validators.url(input_model_url) and "huggingface.co" in input_model_url:
                # Extract the model ID from the URL
                model_id = input_model_url.split("huggingface.co/")[-1]
                model_info = hf_api.model_info(model_id) # Get model info: https://huggingface.co/docs/huggingface_hub/v0.29.2/en/package_reference/hf_api#huggingface_hub.ModelInfo
                model_card = hf.hf_hub_download(models[0].modelId, 'README.md')

                metadata = { # Moved metadata assignment inside the if block
                    "model_id": model_id,
                    "model_info": model_info.__dict__
                }
                return metadata # Return is now inside if block
                json_output = json.dumps(metadata, indent=4, default=str)
                with open('model_info.json', 'w') as json_file:
                 json_file.write(json_output)
                print(json_output)
        else:
            print("Invalid URL. Please enter a valid Hugging Face model URL.")
            # Optionally: You could return None or an empty dictionary here
            # to indicate an invalid URL
# To test: https://huggingface.co/deepseek-ai/DeepSeek-R1

input_url()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 781.3 kB/s eta 0:00:00
Enter model URL: https://huggingface.co/deepseek-ai/DeepSeek-R1
You entered: https://huggingface.co/deepseek-ai/DeepSeek-R1


TypeError: 'generator' object is not subscriptable

# Script 2
Takes in a (validated) model and outputs the children models/fine-tunes

In [ ]:
# Script 2
  # 1. Take link as input (format check). This is the "main model"
  # 2. Give the link to the page with the fine-tunes for the inputted model

from huggingface_hub import HfApi
import requests
from bs4 import BeautifulSoup
import re

# Initialize API
api = HfApi()

# Function to validate Hugging Face model URL
def validate_hf_model_url(url):
    pattern = r"^https://huggingface.co/([\w\-]+)/([\w\-]+)$"
    match = re.match(pattern, url)
    if match:
        return match.groups()  # Returns (org/user, model_name)
    return None

# Function to find fine-tuned models
def get_finetuned_models_page(model_org, model_name):
    search_url = f"https://huggingface.co/models?search={model_name}"
    response = requests.get(search_url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        model_links = [
            a["href"] for a in soup.find_all("a", href=True)
            if model_name.lower() in a["href"].lower()
        ]
        return [f"https://huggingface.co{link}" for link in model_links if model_org not in link]

    return []

# Main execution
model_url = input("Enter the Hugging Face model URL: ").strip()

validated = validate_hf_model_url(model_url)
if validated:
    org, model_name = validated
    finetune_links = get_finetuned_models_page(org, model_name)

    if finetune_links:
        print("Fine-tuned models found:")
        for link in finetune_links:
            print(link)
    else:
        print("No fine-tuned models found for this model.")
else:
    print("Invalid Hugging Face model URL format.")



# Script 3
## Search steps overview
- `dfs_finetunes` we take the `model_url` as input. Alternatively, we can add this var as an argument.
- We go layer-by-layer and find the children of the current model (i.e. the fine-tunes of a model)
- We call the `dfs_funetunes` function recursively and store the models that have been "visited" to avoid duplicates.
- We have a dictionary of information that we store about the "current model" and have the information stored in respective columns (model_id, card, metadata, depth, children (list of model links), children count).
- We have a `results` list that has the information about all the models and their fine-tunes.

In [ ]:
# Script 3
!pip install huggingface_hub
!pip install adapters
!pip install backoff # handle rate-limiting
import requests
import pandas as pd
import datetime
import json
import csv
from huggingface_hub import HfApi
from bs4 import BeautifulSoup
from adapters import list_adapters
from huggingface_hub import hf_hub_download
from adapters import AutoAdapterModel
import re
from backoff import



# Initialize API
api = HfApi()

# Function to validate Hugging Face model URL
def validate_hf_model_url(url):
    pattern = r"^https://huggingface.co/([\w\-]+)/([\w\-]+)$"
    match = re.match(pattern, url)
    return match.groups() if match else None

# page with model finetunes
def get_finetuned_models_page(model_org, model_name):
    all_model_links = []  # Store all links across pages
    page_num = 0
    while True:
        search_url = f"https://huggingface.co/models?other=base_model:finetune:{model_org}/{model_name}&p={page_num}"
        response = requests.get(search_url)
        if response.status_code != 200:
            break  # Exit if page not found

        soup = BeautifulSoup(response.text, "html.parser")
        model_divs = soup.find_all("div", class_="w-full truncate")
        if not model_divs:
            break  # Exit if no more models on the page

        for div in model_divs:
            header = div.find("header")
            if header:
                model_link = header.get("title")
                if model_link:
                    all_model_links.append(f"https://huggingface.co/{model_link}")

        page_num += 1  # Move to the next page

    return all_model_links

# model card data
@backoff.on_exception(backoff.expo,
                      requests.exceptions.RequestException,
                      max_tries=5)
def get_model_card(model_id):
    try:
        # Try to download model card if available
        readme_path = hf_hub_download(repo_id=model_id, filename='README.md')
        with open(readme_path, 'r', encoding='utf-8') as f:
            card_content = f.read()
        return card_content
    except Exception as e:
        print(f"  Could not download model card: {str(e)[:100]}...")
        return ""

# Truncate metadata
def filter_metadata(json_metadata):
            keys_to_keep = ["modelId", "sha", "tags", "downloads", "pipeline_tag"]
            return {k: json_metadata.get(k) for k in keys_to_keep if k in json_metadata}
            filtered_metadata = filter_metadata(api.model_info(model_id).__dict__)

# Get adapter models
@backoff.on_exception(backoff.expo,
                      requests.exceptions.RequestException,
                      max_tries=5)
def get_adapter_models_page(model_org, model_name):
    all_adapter_links = []  # Store all adapter links across pages
    page_num = 0
    while True:
        search_url = f"https://huggingface.co/models?other=base_model:adapter:{model_org}/{model_name}&p={page_num}"
        response = requests.get(search_url)
        if response.status_code != 200:
            break  # Exit if page not found

        soup = BeautifulSoup(response.text, "html.parser")
        model_divs = soup.find_all("div", class_="w-full truncate")
        if not model_divs:
            break  # Exit if no more models on the page

        for div in model_divs:
            header = div.find("header")
            if header:
                model_link = header.get("title")
                if model_link:
                    all_adapter_links.append(f"https://huggingface.co/{model_link}")

        page_num += 1  # Move to the next page

    return all_adapter_links

# Recursive DFS (depth-first search) for finding fine-tunes
@backoff.on_exception(backoff.expo,
                      (requests.exceptions.RequestException,
                       huggingface_hub.utils.HfHubHTTPError),
                      max_tries=5)
def dfs_finetunes(model_url, visited, depth=0, results=None):
       if results is None:
           results = []

       if model_url in visited:
           return results
       visited.add(model_url)

       validated = validate_hf_model_url(model_url)
       if not validated:
           print(f"Invalid URL skipped: {model_url}")
           model_url = "N/A"
           return results

       model_org, model_name = validated
       model_id = f"{model_org}/{model_name}"


       print(f"\n{'  ' * depth}Fetching metadata for: {model_id}")
       try:
           model_metadata = api.model_info(model_id).__dict__
           json_metadata = json.dumps(model_metadata, default=str)
           model_card = get_model_card(model_id)

       except Exception as e:
           print(f"Error fetching metadata: {e}")
           return results

       finetune_links = get_finetuned_models_page(model_org, model_name)
       # Removing Duplicate Children
       finetune_links = list(set(finetune_links))
       print(f"{'  ' * depth}Found {len(finetune_links)} fine-tunes at depth {depth}.")
       adapter_links = get_adapter_models_page(model_org, model_name)
       print(f"{'  ' * depth}Found {len(adapter_links)} adapter models for {model_id}.")

       results.append({
           "model_id": model_id,
           "card": model_card,
           "metadata": json_metadata,
           "depth": depth,
           "children": finetune_links,
           "children_count": len(finetune_links),
           "adapters": adapter_links,
           "adapters_count": len(adapter_links)
       })

       for link in finetune_links:
             dfs_finetunes(link, visited, depth + 1, results)
       return results

# Timestamp for the run
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Function to save results as JSON
def save_json(results, model_name):
    filename = f"{model_name}_finetunes_{timestamp}.json"
    data = {
        "models": results
    }
    with open(filename, "w") as f:
        json.dump(data, f, indent=4, default=str)
    print(f"Results saved to {filename}")

# Function to save results as CSV
''' def save_csv(results, model_name):
    filename = f"{model_name}_{timestamp}_finetunes.csv"
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["model_id", "depth", "children_count", "children", "metadata"])
        writer.writeheader()
        for entry in results:
            # Ensure metadata is a JSON string
            if isinstance(entry["metadata"], dict):
                entry["metadata"] = json.dumps(entry["metadata"], indent=2, default=str)
            # Join children list as a string
            entry["children"] = ", ".join(entry["children"])
            writer.writerow(entry)
    print(f"Results saved to {filename}") '''

# Function to save results as CSV (pandas)
def save_csv(results, model_name):
    filename = f"{model_name}_finetunes_{timestamp}.csv"
    df = pd.DataFrame(results)
    df.to_csv(filename, index=True)
    print(f"Results saved to {filename}")

# Main execution
if __name__ == "__main__":
    model_url = input("Enter the Hugging Face model URL: ").strip()
    visited = set()
    results = dfs_finetunes(model_url, visited)

    if results:
        model_name = results[0]["model_id"].split("/")[-1]  # Extract model name for file naming
        save_json(results, model_name)
        save_csv(results, model_name)
    else:
        print("No fine-tuned models found.")

'''Links for testing: https://huggingface.co/NousResearch/DeepHermes-3-Mistral-24B-Preview (3 fine-tunes at depth 0, 1 fine-tune at depth 1 for 'AlSamCur123/DeepHermes-3-Mistral-24BContinuedFine')
https://huggingface.co/perplexity-ai/r1-1776 (11 fine-tunes at depth 0)'''

# Scripts 4 (Tree Viz)

Iteratively produces model tree datasets when given a list of models.

In [ ]:
# 4.1 Phylogenetic tree

!pip install Bio
!pip install pandas
import pandas as pd
import ast
import matplotlib.pyplot as plt
from Bio import Phylo
from Bio.Phylo.BaseTree import Clade, Tree

# Load and prepare tree
df = pd.read_csv("/content/DeepSeek-R1_finetunes_20250408_202441.csv")
df['children'] = df['children'].apply(ast.literal_eval)

def build_phylo_tree_from_dfs(df):
    clade_map = {row['model_id']: Clade(name=row['model_id']) for _, row in df.iterrows()}
    parent_links = {}
    for _, row in df.iterrows():
        parent = row['model_id']
        for child_url in row['children']:
            child = '/'.join(child_url.split("/")[-2:])
            parent_links[child] = parent
    all_models = set(df['model_id'])
    child_models = set(parent_links.keys())
    root_model_id = list(all_models - child_models)[0]
    for child, parent in parent_links.items():
        if child in clade_map and parent in clade_map:
            clade_map[parent].clades.append(clade_map[child])
    return Tree(root=clade_map[root_model_id])

tree = build_phylo_tree_from_dfs(df)

# Plotting with labels on branches
fig = plt.figure(figsize=(24, 36))
ax = fig.add_subplot(1, 1, 1)

# Draw the tree without axes
Phylo.draw(tree, axes=ax, do_show=False)

# Remove all axes elements
ax.set_axis_off()
ax.set_xticks([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

# Ensure labels are positioned away from branch lines
for label in ax.texts:
    pos = label.get_position()
    label.set_position((pos[0], pos[1]))
    label.set_bbox(dict(facecolor='white', alpha=0.8, edgecolor='none', pad=3))

plt.tight_layout()
plt.savefig('clean_phylogenetic_tree.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# 4.1 Networkx (in progress)
!apt install graphviz libgraphviz-dev
!pip install pygraphviz
import pandas as pd
import ast
import matplotlib.pyplot as plt
import networkx as nx

# Load and prepare the dataset
df = pd.read_csv("/content/DeepSeek-R1_finetunes_20250408_202441.csv")
df['children'] = df['children'].apply(ast.literal_eval)

# Create a directed graph
G = nx.DiGraph()

# Add nodes and edges
for _, row in df.iterrows():
    parent = row['model_id']
    G.add_node(parent)
    for child_url in row['children']:
        child = '/'.join(child_url.split("/")[-2:])
        G.add_node(child)
        G.add_edge(parent, child)

# Find root node
root = [n for n, d in G.in_degree() if d == 0][0]

# Create a hierarchical layout
pos = nx.nx_agraph.graphviz_layout(G, prog='dot', root=root)

# Create figure
plt.figure(figsize=(16, 24))

# Draw nodes
nx.draw_networkx_nodes(G, pos, node_size=10, node_color='white', edgecolors='black')

# Draw edges
nx.draw_networkx_edges(G, pos, arrows=False)

# Draw labels, rotating only non-root nodes
label_pos = {k: (v[0], v[1]) for k, v in pos.items()}
for node, (x, y) in label_pos.items():
    if node != root:  # Rotate labels for non-root nodes
        plt.text(x, y, s=node, rotation=90, ha='center', va='center', fontsize=9)
    else:  # Keep root label horizontal
        plt.text(x, y, s=node, ha='center', va='center', fontsize=9)

# Remove axes
plt.axis('off')
plt.tight_layout()

# Save the figure
plt.savefig("mistral_finetune_tree_networkx.png", dpi=150, bbox_inches='tight')
plt.show()

print("Tree visualization saved to mistral_finetune_tree_networkx.png")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6ubuntu0.1).
The following additional packages will be installed:
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common
  libgvc6-plugins-gtk librsvg2-common libxdot4
Suggested packages:
  gvfs
The following NEW packages will be installed:
  libgail-common libgail18 libgraphviz-dev libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libgvc6-plugins-gtk librsvg2-common libxdot4
0 upgraded, 9 newly installed, 0 to remove and 30 not upgraded.
Need to get 2,434 kB of archives.
After this operation, 7,681 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libgtk2.0-common all 2.24.33-2ubuntu2.1 [125 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libgtk2.0-0 amd64 2.24.33-2ubuntu2.1 [2,038 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libgail18 

FileNotFoundError: [Errno 2] No such file or directory: '/content/DeepSeek-R1_finetunes_20250408_202441.csv'

In [42]:
# 4.3 library (in progress)
!pip install ete3
!pip install tree
!pip install PyQt5
import pandas as pd
import ast
from ete3 import Tree, faces, AttrFace, TreeStyle


# Load and prepare the dataset
df = pd.read_csv("/content/DeepSeek-R1_finetunes_20250408_202441.csv")
df['children'] = df['children'].apply(ast.literal_eval)

# Function to build the ete3 tree recursively
def build_ete3_tree(df, parent_node=None, parent_name=None):
    if parent_node is None:
        # Create the root node if it's the first call
        root_name = df.loc[df['depth'] == 0, 'model_id'].iloc[0]
        tree = Tree(name=root_name)
        parent_node = tree
    else:
        tree = parent_node  # Continue building the existing tree

    # Find children of the current parent
    children_rows = df.loc[df['model_id'] == parent_name]
    if not children_rows.empty:
        children_urls = children_rows['children'].iloc[0]
        for child_url in children_urls:
            child_name = '/'.join(child_url.split("/")[-2:])
            child_node = tree.add_child(name=child_name)
            build_ete3_tree(df, child_node, child_name)  # Recursive call
    return tree

# Build the tree
tree = build_ete3_tree(df)

# Style the tree
ts.show_leaf_name = False  # Hide leaf names
ts.mode = "c"  # Circular tree layout
ts.root_opening_factor = 1  # Adjust root opening for circular layout

# Style the nodes
nst = NodeStyle()
nst["size"] = 0  # Hide node circles
nst["hz_line_width"] = 2
nst["vt_line_width"] = 2

for n in tree.traverse():
    n.set_style(nst)
    # Add model name as a branch label
    name_face = TextFace(n.name, fsize=10, fgcolor="black")
    n.add_face(name_face, column=0, position="branch-right")

# Render and save the tree
tree.render("mistral_finetune_tree_ete3.png", tree_style=ts, dpi=300)
print("Tree visualization saved to mistral_finetune_tree_ete3.png")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 20.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ete3: filename=ete3-3.1.3-py3-none-any.whl size=2273786 sha256=11e90a6cb410702e112dd5bb1b8cef8bb0a9090cbb05bb8d9a45eb80da1f6665
  Stored in directory: /root/.cache/pip/wheels/dd/a8/60/0a29caa9f8ceb7316704be63c1578ab13c36668abb646366ac
Successfully built ete3
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 1.1 MB/s eta 0:00:00
  Created wheel for tree: filename=Tree-0.2.4-py3-none-any.whl size=7860 sha256=59cd350907ddf5fc946a6284821b20adcb2e8efbe4421508eb98085447f94f41
  Stored in directory: /root/.cache/pip/wheels/e8/ed/fe/b4c6a9b7a5b8df6d966ea673e26a46a7451b020af754eafa6b
Successfully built tree
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/

FileNotFoundError: [Errno 2] No such file or directory: '/content/DeepSeek-R1_finetunes_20250408_202441.csv'

# Script 6

Get top 1000 models from the model-hub ordered by the number of likes

In [ ]:
!pip install huggingface_hub
import pandas as pd
import datetime
import csv
from huggingface_hub import HfApi

api = HfApi()
time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
# documentation: https://huggingface.co/docs/hub/en/api, gets top 1000 models sorted by likes in descending order
models = api.list_models(sort="likes", direction="-1", limit=1000)

df = pd.DataFrame(models)
models_file = df.to_csv(f"{time}_model_list.csv", index="false")
print("Models saved to CSV")


# Script 7

This script allows you to iteratively run the dfs function for multiple models given as a CSV

In [ ]:
# Multi-model runner for HuggingFace model analysis (sampled from top 10 models)
import requests
import pandas as pd
import datetime
import json
import csv
from huggingface_hub import HfApi
from bs4 import BeautifulSoup
from adapters import list_adapters
from huggingface_hub import hf_hub_download
from adapters import AutoAdapterModel
import re

# Multi-model runner for HuggingFace model analysis (sampled from top 10 models)
import requests
import pandas as pd
import datetime
import json
import csv
import os
import time
from huggingface_hub import HfApi
from bs4 import BeautifulSoup
import re


# Make sure these are imported or defined in the same file or via `import script3`
#from /content/script3.py import dfs_finetunes, save_json, save_csv  # Import from script3.py
# Assuming script3.py is in the same directory, and dfs_finetunes, save_json, and save_csv are defined there
from script3 import dfs_finetunes, save_json, save_csv


# Timestamp to share across all model saves
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

def process_models_from_csv(csv_path, output_dir="results", limit=10):
    """
    Processes top N models from a CSV by calling `dfs_finetunes` for each.
    Constructs model URLs from the 'id' column and saves results in JSON/CSV.
    """
    os.makedirs(output_dir, exist_ok=True)
    df = pd.read_csv(csv_path)

    if 'id' not in df.columns:
        raise ValueError("CSV must contain a column named 'id'.")

    df = df.head(limit)
    total_models = len(df)

    for idx, row in df.iterrows():
        model_id = row['id'].strip()
        model_url = f"https://huggingface.co/{model_id}"
        print(f"\n[{idx + 1}/{total_models}] Running analysis for: {model_url}")

        try:
            visited = set()
            results = dfs_finetunes(model_url, visited)

            if results:
                model_name = model_id.split("/")[-1]
                json_path = os.path.join(output_dir, f"{model_name}_finetunes_{timestamp}.json")
                csv_path = os.path.join(output_dir, f"{model_name}_finetunes_{timestamp}.csv")

                save_json(results, model_name)
                save_csv(results, model_name)

                print(f"Completed: {model_name}")
            else:
                print(f"No results for {model_id}")

            time.sleep(2)  # Rate limit buffer

        except Exception as e:
            print(f"Error with {model_id}: {e}")

    print(f"\nFinished processing {total_models} models. Saved to '{output_dir}'.")

if __name__ == "__main__":
    csv_input = input("Path to CSV with model IDs: ").strip()
    process_models_from_csv(csv_input)
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

def process_models_from_csv(csv_path, output_dir="results", limit=10):
    """
    Processes top N models from a CSV by calling `dfs_finetunes` for each.
    Constructs model URLs from the 'id' column and saves results in JSON/CSV.
    """
    os.makedirs(output_dir, exist_ok=True)
    df = pd.read_csv(csv_path)

    if 'id' not in df.columns:
        raise ValueError("CSV must contain a column named 'id'.")

    df = df.head(limit)
    total_models = len(df)

    for idx, row in df.iterrows():
        model_id = row['id'].strip()
        model_url = f"https://huggingface.co/{model_id}"
        print(f"\n[{idx + 1}/{total_models}] Running analysis for: {model_url}")

        try:
            visited = set()
            results = dfs_finetunes(model_url, visited)

            if results:
                model_name = model_id.split("/")[-1]
                json_path = os.path.join(output_dir, f"{model_name}_finetunes_{timestamp}.json")
                csv_path = os.path.join(output_dir, f"{model_name}_finetunes_{timestamp}.csv")

                save_json(results, model_name)
                save_csv(results, model_name)

                print(f"Completed: {model_name}")
            else:
                print(f"No results for {model_id}")

            time.sleep(2)  # Rate limit buffer

        except Exception as e:
            print(f"Error with {model_id}: {e}")

    print(f"\nFinished processing {total_models} models. Saved to '{output_dir}'.")

if __name__ == "__main__":
    csv_input = input("Path to CSV with model IDs: ").strip()
    process_models_from_csv(csv_input)

